#### 1. Download and Install Rclone

In [ ]:
!wget https://downloads.rclone.org/v1.56.0/rclone-v1.56.0-linux-amd64.deb
!apt install ./rclone-v1.56.0-linux-amd64.deb

#### 2. Setup Rclone config

In [ ]:
!rclone config

#### 3. Mount OneDrive

In [ ]:
!nohup rclone --vfs-cache-mode writes mount onedrive: ./OneDrive &

#### 4. Mount Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

#### 5. Copy files

In [ ]:
# ============================= FORM ============================= #
# @markdown #### Execute rClone

Mode = "Copy"  # @param ["Move", "Copy", "Sync", "Verify", "Dedupe", "Clean Empty Dirs", "Empty Trash"]
Source = ""  # @param {type:"string"}
Destination = ""  # @param {type:"string"}
Extra_Arguments = ""  # @param {type:"string"}
COPY_SHARED_FILES = False  # @param{type: "boolean"}
Compare = "Size & Checksum"
TRANSFERS, CHECKERS = 20, 20
THROTTLE_TPS = True
BRIDGE_TRANSFER = False  # @param{type: "boolean"}
FAST_LIST = False  # @param{type: "boolean"}
OPTIMIZE_GDRIVE = True
SIMPLE_LOG = True
RECORD_LOGFILE = True  # @param{type: "boolean"}
SKIP_NEWER_FILE = False
SKIP_EXISTED = False
SKIP_UPDATE_MODTIME = False
ONE_FILE_SYSTEM = False
LOG_LEVEL = "DEBUG"
SYNC_MODE = "Delete after transfering"
SYNC_TRACK_RENAME = True
DEDUPE_MODE = "Largest"
USE_TRASH = True
DRY_RUN = False  # @param{type: "boolean"}
# ================================================================ #
from os import path as _p

if not _p.exists("/root/.ipython/rlab_utils.py"):
    from shlex import split as _spl
    from subprocess import run

    shellCmd = "wget -qq https://biplobsd.github.io/RLabClone/res/rlab_utils.py \
                    -O /root/.ipython/rlab_utils.py"
    run(_spl(shellCmd))

from datetime import datetime as _dt
from rlab_utils import (
    displayOutput,
    checkAvailable,
    runSh,
    prepareSession,
    PATH_RClone_Config,
    accessSettingFile,
    memGiB,
)


def populateActionArg():
    if Mode == "Copy":
        actionArg = "copy"
    elif Mode == "Sync":
        actionArg = "sync"
    elif Mode == "Verify":
        actionArg = "check"
    elif Mode == "Dedupe":
        actionArg = "dedupe largest"
    elif Mode == "Clean Empty Dirs":
        actionArg = "rmdirs"
    elif Mode == "Empty Trash":
        actionArg = "delete"
    else:
        actionArg = "move"

    return actionArg


def populateCompareArg():
    if Compare == "Mod-Time":
        compareArg = "--ignore-size"
    elif Compare == "Size":
        compareArg = "--size-only"
    elif Compare == "Checksum":
        compareArg = "-c --ignore-size"
    else:
        compareArg = "-c"

    return compareArg


def populateOptimizeGDriveArg():
    return (
        "--buffer-size 256M \
            --drive-chunk-size 256M \
                --drive-upload-cutoff 256M \
                    --drive-acknowledge-abuse \
                        --drive-keep-revision-forever"
        if OPTIMIZE_GDRIVE
        else "--buffer-size 128M"
    )


def populateGDriveCopyArg():
    if BRIDGE_TRANSFER and memGiB() < 13:
        global TRANSFERS, CHECKERS
        TRANSFERS, CHECKERS = 10, 80
    else:
        pass
    return "--disable copy" if BRIDGE_TRANSFER else "--drive-server-side-across-configs"


def populateStatsArg():
    statsArg = "--stats-one-line --stats=5s" if SIMPLE_LOG else "--stats=5s -P"
    if LOG_LEVEL != "OFF":
        statsArg += " -v" if SIMPLE_LOG else "-vv"
    elif LOG_LEVEL == "INFO":
        statsArg += " --log-level INFO"
    elif LOG_LEVEL == "ERROR":
        statsArg += " --log-level ERROR"
    else:
        statsArg += " --log-level DEBUG"
    return statsArg


def populateSyncModeArg():
    if Mode != "Sync":
        return ""
    elif SYNC_MODE == "Delete before transfering":
        syncModeArg = "--delete-before"
    elif SYNC_MODE == "Delete after transfering":
        syncModeArg = "--delete-after"
    else:
        syncModeArg = "--delete-during"
    if SYNC_TRACK_RENAME:
        syncModeArg += " --track-renames"
    return syncModeArg


def populateDedupeModeArg():
    if DEDUPE_MODE == "Interactive":
        dedupeModeArg = "--dedupe-mode interactive"
    elif DEDUPE_MODE == "Skip":
        dedupeModeArg = "--dedupe-mode skip"
    elif DEDUPE_MODE == "First":
        dedupeModeArg = "--dedupe-mode first"
    elif DEDUPE_MODE == "Newest":
        dedupeModeArg = "--dedupe-mode newest"
    elif DEDUPE_MODE == "Oldest":
        dedupeModeArg = "--dedupe-mode oldest"
    elif DEDUPE_MODE == "Rename":
        dedupeModeArg = "--dedupe-mode rename"
    else:
        dedupeModeArg = "--dedupe-mode largest"

    return dedupeModeArg


def generateCmd():
    sharedFilesArgs = (
        "--drive-shared-with-me --files-from /content/upload.txt --no-traverse"
        if COPY_SHARED_FILES
        else ""
    )

    logFileArg = f"--log-file /content/rclone_log.txt -vv -P"

    args = [
        "rclone",
        f"--config {PATH_RClone_Config}/rclone.conf",
        '--user-agent "Mozilla"',
        populateActionArg(),
        f'"{Source}"',
        f'"{Destination}"' if Mode in ("Move", "Copy", "Sync") else "",
        f"--transfers {str(TRANSFERS)}",
        f"--checkers {str(CHECKERS)}",
    ]

    if Mode == "Verify":
        args.append("--one-way")
    elif Mode == "Empty Trash":
        args.append("--drive-trashed-only --drive-use-trash=false")
    else:
        args.extend(
            [
                populateGDriveCopyArg(),
                populateSyncModeArg(),
                populateCompareArg(),
                populateOptimizeGDriveArg(),
                "-u" if SKIP_NEWER_FILE else "",
                "--ignore-existing" if SKIP_EXISTED else "",
                "--no-update-modtime" if SKIP_UPDATE_MODTIME else "",
                "--one-file-system" if ONE_FILE_SYSTEM else "",
                "--tpslimit 95 --tpslimit-burst 40" if THROTTLE_TPS else "",
                "--fast-list" if FAST_LIST else "",
                "--delete-empty-src-dirs" if Mode == "Move" else "",
            ]
        )
    args.extend(
        [
            "-n" if DRY_RUN else "",
            populateStatsArg() if not RECORD_LOGFILE else logFileArg,
            sharedFilesArgs,
            Extra_Arguments,
        ]
    )

    return args


def executeRclone():
    prepareSession()
    if Source.strip() == "":
        displayOutput("❌ The Source field is empty.")
        return
    if checkAvailable("/content/rclone_log.txt"):
        if not checkAvailable("/content/logfiles"):
            runSh("mkdir -p -m 666 /content/logfiles")
        job = accessSettingFile("job.txt")
        runSh(
            f'mv /content/rclone_log.txt /content/logfiles/{job["title"]}_{job["status"]}_logfile.txt'
        )

    onGoingJob = {
        "title": f'{Mode}_{Source}_{Destination}_{_dt.now().strftime("%a-%H-%M-%S")}',
        "status": "ongoing",
    }
    accessSettingFile("job.txt", onGoingJob)

    cmd = " ".join(generateCmd())
    runSh(cmd, output=True)
    displayOutput(Mode, "success")

    onGoingJob["status"] = "finished"
    accessSettingFile("job.txt", onGoingJob)

executeRclone()
